<a href="https://colab.research.google.com/github/rzylius/household-battery-dispatch/blob/main/household_battery_dispatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 73.2 MB/s eta 0:00:00


In [7]:
# Retrieve Nordpool prices of LT region and form a list
import requests

URL = "http://np-lt-day-ahead-pricing.azurewebsites.net/api/date/today"
#URL = "http://np-lt-day-ahead-pricing.azurewebsites.net/api/date/tomorrow"

np = requests.get(URL)
d=np.json()
d.pop('metadata')
data = sorted(d.items(), key=lambda x: x[1])

d_normal = d

p = []
for i in d:
  p.append(d[i])
print(p)

[16.17, 15.62, 15.01, 14.6, 16.01, 16.95, 21.92, 26.49, 26.49, 24.33, 22.6, 22.0, 21.28, 20.87, 21.83, 23.7, 27.01, 27.86, 28.32, 28.67, 27.51, 20.99, 18.71, 16.62]


In [5]:
import pulp

def optimize_power_consumption(battery_capacity,
                               hourly_prices,
                               initial_soc,
                               target_soc,
                               hourly_consumption,
                               kwh_cycle_cost,
                               max_charge_power):

    # Create a linear programming problem
    problem = pulp.LpProblem("Power_Optimization", pulp.LpMinimize)

    # Define decision variables
    hours = range(len(hourly_prices))

    command = pulp.LpVariable.dicts("Command", hours, lowBound = 0, upBound= max_charge_power + hourly_consumption)
    soc = pulp.LpVariable.dicts("SOC", hours, lowBound=0, upBound=battery_capacity)

    # Define the objective function to minimize cost
    # TODO: debug hot to add costs of cycle to equation
    problem += pulp.lpSum(hourly_prices[hour] * command[hour] for hour in hours) + (pulp.lpSum(command[hour] for hour in hours) - hourly_consumption)*kwh_cycle_cost

    # Define constraints
    for hour in hours:
      if hour == 0: init_soc = initial_soc
      else: init_soc = soc[hour -1]
      problem += soc[hour] == init_soc - hourly_consumption + command[hour]
      problem += soc[hour] <= battery_capacity # battery soc does not exceed capacity
      problem += soc[hour] >= 2 # battery soc should not go lower than that

    # Ensure that the target SOC is achieved at the end
    problem += soc[23] == target_soc

    # Solve the linear programming problem
    status = problem.solve()



    # Extract the optimal solution
    optimization_result = {
        "Hour": [],
        "Command": [],
        "Projected_SOC": []
    }

    optimization_res = []

    print(f"Optimization produced result: {pulp.LpStatus[status]}")
    print()
    print("Battery commands")
    print(f"Initial SOC {initial_soc}")
    print("Hour - kWh price - Command - SOC - Grid consumption - No opt price - Opt price")

    for hour in hours:
        optimization_result["Hour"].append(hour)
        optimization_result["Command"].append("Charge" if command[hour].varValue > (hourly_consumption+0.1) else "Discharge" if command[hour].varValue < (hourly_consumption-0.1) else "Idle")
        optimization_result["Projected_SOC"].append(soc[hour].varValue)

        r = []
        r.append(hour)
        r.append(round(hourly_prices[hour],2))
        r.append("Charge   " if command[hour].varValue > (hourly_consumption+0.1) else "Discharge" if command[hour].varValue < (hourly_consumption-0.1) else "Idle     ")
        r.append(command[hour].varValue)
        r.append(soc[hour].varValue)
        r.append(round(hourly_prices[hour] * hourly_consumption ,2))
        r.append(round(hourly_prices[hour] * command[hour].varValue ,2))
        optimization_res.append(r)
        print(r)


    return optimization_res

# Example usage:
battery_capacity = 15  # kWh
#hourly_prices = p
hourly_prices = [15, 18, 19, 10, 10, 10, 10, 10, 10, 10, 10, 12, 14, 15, 15, 13, 14, 12, 15, 14, 13, 11, 10, 13]
initial_soc = 10  # kWh
target_soc = 10  # kWh
hourly_consumption = 1.2 # kwh the house consumes approx every hour
cost_of_cycle = 5 # cents it costs per charge/discharge cycle of the battery (depreciacion costs)
max_charge_power = 5 # maximum power in KW that battery can be charged with

result = optimize_power_consumption(battery_capacity, hourly_prices, initial_soc, target_soc, hourly_consumption, cost_of_cycle, max_charge_power)

print()
print()
print("Optimization results")

print(f"No optimization. Total {sum([i[3] for i in result]):.2f} kwh. Avg {(sum([i[5] for i in result])/len(result)):.2f} ct/kwh. Total bill {sum([i[5] for i in result])/100:.2f} EUR")
print(f"Po optimizavimo. Total {sum([i[3] for i in result]):.2f} kwh. Avg {sum([i[6] for i in result])/len(result):.2f} ct/kwh. Total bill {sum([i[6] for i in result])/100:.2f} EUR")



Optimization produced result: Optimal

Battery commands
Initial SOC 10
Hour - kWh price - Command - SOC - Grid consumption - No opt price - Opt price
[0, 15, 'Discharge', 0.0, 8.8, 18.0, 0.0]
[1, 18, 'Discharge', 0.0, 7.6, 21.6, 0.0]
[2, 19, 'Discharge', 0.0, 6.4, 22.8, 0.0]
[3, 10, 'Discharge', 0.0, 5.2, 12.0, 0.0]
[4, 10, 'Discharge', 0.0, 4.0, 12.0, 0.0]
[5, 10, 'Discharge', 0.0, 2.8, 12.0, 0.0]
[6, 10, 'Discharge', 0.4, 2.0, 12.0, 4.0]
[7, 10, 'Idle     ', 1.2, 2.0, 12.0, 12.0]
[8, 10, 'Charge   ', 4.2, 5.0, 12.0, 42.0]
[9, 10, 'Charge   ', 6.2, 10.0, 12.0, 62.0]
[10, 10, 'Charge   ', 6.2, 15.0, 12.0, 62.0]
[11, 12, 'Discharge', 0.0, 13.8, 14.4, 0.0]
[12, 14, 'Discharge', 0.0, 12.6, 16.8, 0.0]
[13, 15, 'Discharge', 0.0, 11.4, 18.0, 0.0]
[14, 15, 'Discharge', 0.0, 10.2, 18.0, 0.0]
[15, 13, 'Discharge', 0.0, 9.0, 15.6, 0.0]
[16, 14, 'Discharge', 0.0, 7.8, 16.8, 0.0]
[17, 12, 'Discharge', 0.0, 6.6, 14.4, 0.0]
[18, 15, 'Discharge', 0.0, 5.4, 18.0, 0.0]
[19, 14, 'Discharge', 0.0, 4.2, 1